# Package that we need

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os
from sklearn import svm
from sklearn.decomposition import PCA
from scipy.stats import multivariate_normal


# you can choose one of the following package for image reading/processing

import cv2
import PIL


## 1 Support Vector Machine

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

In the training procedure of SVM, we need to optimize with respect to the Lagrange multipliers
$a = \{ a_n \}$. 

Here, we use the <span class="blue">sequential minimal optimization</span> to solve the problem. 

For details, you can refer to the paper [Platt, John. “Sequential minimal optimization: 

A fast algorithm for training support vector machines”, 1998]. The classifier is written by

$$
    y(\textbf{x}) = \sum_{n=1}^N a_n t_n k(\textbf{x}, \textbf{x}_n) = \textbf{w}^T \textbf{x} + b
$$
$$
    \textbf{w} = \sum_{n=1}^N \alpha_n t_n \phi(\textbf{x}_n)
$$
$$
    b = \frac{1}{N_{\mathcal{M}}} \sum_{n \in \mathcal{M}} \left( t_n - \sum_{m \in \mathcal{S}} a_m t_m k(\textbf{x}_n, \textbf{x}_m) \right)
$$

where $\mathcal{M}$ denotes the set of indices of data points having $0 \lt a_n \lt C$.

### 1.1

It is popular to use principal component analysis (PCA) to reduce the dimension of images to d = 2. 

Please implement it by yourself instead of using the method from sklearn.

In [2]:
data = pd.read_csv("./x_train.csv",header= None)/255
label = pd.read_csv("./t_train.csv",header= None)

**PCA**

$\textbf{C}$ is covariance matrix of $\textbf{X}$, $\textbf{W}$ is the eigenvector matrix of $\textbf{C}$, $\Lambda$ is the eigenvalue matrix of $\textbf{C}$.

$\textbf{X}$ is the data matrix, $\textbf{X}_k$ is the data matrix after dimension reduction, and k in this task will be 2.

$$
    \textbf{C} = \frac{\textbf{X}^T \textbf{X}}{n - 1}
$$
$$
    \textbf{C} = \textbf{W} \Lambda \textbf{W}^{-1}
$$
$$
    \textbf{X}_k = \textbf{X} \textbf{W}_k
$$

**Using SVD to solve PCA**

$\Sigma$ is the singular value matrix of $\textbf{X}$, $\textbf{U}$ is the left singular vector matrix of $\textbf{X}$, $\textbf{V}$ is the right singular vector matrix of $\textbf{X}$.

$$
    \textbf{C} = \frac{\textbf{X}^T \textbf{X}}{n - 1} 
$$
$$
    \textbf{C} = \frac{\textbf{V}\Sigma\textbf{U}^T\textbf{U}\Sigma\textbf{V}^T}{n - 1}
$$
$$
    \textbf{C} = \textbf{V} \frac{\Sigma^2}{n - 1}\textbf{V}^T
$$
$$
    \textbf{C} = \textbf{V} \frac{\Sigma^2}{n - 1}\textbf{V}^{-1} \ \ (\rm{because} \ \textbf{V} \ \rm{is \ unitary})
$$

We can found that covariance matrix $\textbf{C}$ is the same as the eigenvalue matrix $\Lambda$ of $\textbf{V}^{-1}$.

$$
    \Lambda = \frac{\Sigma^2}{n - 1}
$$

In [29]:
data_new = data.values - np.mean(data.values, axis=0) # centralize data
np.allclose(data_new.mean(axis=0), np.zeros(data_new.shape[1])) # check if centralization is correct
C = (data_new.T @ data_new) / (data_new.shape[0] - 1) # covariance matrix
eig_vals, eig_vecs = np.linalg.eig(C) # eigenvalues and eigenvectors
data_pca = data_new @ eig_vecs # PCA
U, Sigma, V = np.linalg.svd(data_new, full_matrices=False, compute_uv=True) # SVD
data_pca_svd = data_new @ V.T # PCA by SVD
data_pca_svd = data_pca_svd[:,0:2] # reduce dimension to 2
data_pca_svd # 2D data

array([[-5.99097813e-01, -3.89702805e+00],
       [-5.09292853e+00,  1.62755542e+00],
       [ 2.93357257e+00,  7.81060497e-01],
       [ 7.34954212e+00,  2.67207805e+00],
       [ 8.68487609e+00,  4.52386768e+00],
       [ 1.78352031e+00, -3.72584325e+00],
       [-5.71393538e+00,  2.27382542e+00],
       [-5.99472259e+00,  2.11847433e+00],
       [ 1.28782520e+00, -4.75297021e+00],
       [ 3.49997556e+00, -5.19901731e+00],
       [ 3.95543251e+00, -4.04583782e+00],
       [ 7.73303696e+00,  1.47002267e+00],
       [ 2.84462448e+00, -4.03633020e+00],
       [-5.58473438e+00,  1.66806050e+00],
       [-5.72104940e-01, -4.06527665e+00],
       [ 3.86638527e+00, -4.11302300e+00],
       [ 7.51912680e+00,  3.27160543e+00],
       [ 2.91912021e+00, -5.01469493e+00],
       [-3.84794261e+00,  2.79142344e+00],
       [ 6.98834615e-01, -4.60895089e+00],
       [ 2.89830733e+00, -2.48968641e+00],
       [ 3.09527120e+00, -4.81583703e+00],
       [-6.56076017e+00,  2.80196744e-02],
       [-2.

#### 1.2

Describe the difference between two decision approaches (one-versus-the-rest and one-
versus-one). 

Decide which one you want to choose and explain why you choose this approach.

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

**One-versus-the-rest**

First, we train a binary classifier <span class="red">for each class</span>. 

Only one class is positive and the rest are negative.

Then, we use the classifier to predict the class of the test data. 

Finally, we choose the class with the highest score.

**One-versus-one**

We train a binary classifier <span class="red">for each pair of classes</sapn>. 

Then, we use the classifier to predict the class of the test data. 

The test data is assigned to the class that wins the most duels.

Finally, we choose the class with the highest score.

**Which one to choose**

Support vector machine is a binary classifier.

If we use one-versus-the-rest, we need to train $C$ binary classifiers.

If we use one-versus-one, we need to train $\frac{C(C-1)}{2}$ binary classifiers.

If $C$ is large, <span class = "red">one-versus-one is better</sapn>.

#### 1.3

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

Use the principle values projected to top <span class="blue">two</span> eigenvectors obtained from PCA, and build
a SVM with <span class="blue">linear kernel</span> to do multi-class classification. 

You can decide the upper bound $C$ of $a_n$ by yourself or just use the default value provided by sklearn. 

Then, <span class="blue">plot the corresponding decision boundary</span> and show the <span class="blue">support vectors.</span>

* Linear kernel:
$$
    k(\textbf{x}_i, \textbf{x}_j) = \phi(\textbf{x}_i)^T \phi(\textbf{x}_j) = \textbf{x}_i^T \textbf{x}_j
$$

The sample figures are provided below.

<center>
    <img src = "./image/figure1.png" width = 50%>
</center>

#### Bonus

<style>
.blue{
    color: skyblue;
}
.bold{
    font-weight: bold;
}
</style>

Repeat 3 with <span class="blue">polynomial kernel (degree = 2).</span>

* Polynomial (homogeneous) kernel of degree 2:

$$
    k(\textbf{x}_i, \textbf{x}_j) = \phi(\textbf{x}_i)^T \phi(\textbf{x}_j) = \left( \textbf{x}_i^T \textbf{x}_j\right)^2
$$
$$
    \phi(\textbf{x}) = \left[ \textbf{x}_1^2, \sqrt{2} \textbf{x}_1 \textbf{x}_2, \textbf{x}_2^2 \right]
$$
$$
    \textbf{x} = \left[ x_1, x_2 \right]
$$

## 2 Gaussian Mixture Model

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

In this exercise, you will implement a Gaussian mixture model (GMM) and apply it in image segmentation. 

First, use a $K$-means algorithm to find $K$ central pixels. 

Second, use the expectation maximization (EM) algorithm <span class="blue">(please refer to textbook p.438-p.439)</span> to optimize the parameters of the model. 

The input image is given by <span class="red">hw3.jpg</span>. 

According to the maximum likelihood, you can decide the color $\mu_k$ , $k \in [1, . . . , K]$ of each pixel $x_n$ of output image

#### 2.1

Please build a $K$-means model by minimizing

$$
    J = \sum_{n=1}^N \sum_{k=1}^K \gamma_{nk} \| x_n - \mu_k \|^2
$$

and show the table of the estimated $\{ \mu_k \}^K_{k=1}$.

In [ ]:
image = cv2.imread("./hw3.jpg")

#### 2.2

<style>
.blue{
    color: skyblue;
}
.red{
    color: red;
}
.bold{
    font-weight: bold;
}
</style>

Use $ \mu = \{ \mu_k \}^K_{k=1}$ calculated by the $K$-means model as the means, and calculate the corresponding variances $ \sigma_k^2 $ 

and mixing coefficient $\pi_k$ for the initialization of the GMM $p(x) = \sum_{k=1}^K \pi_k \mathcal{N}(x \ | \ \mu_k, \sigma_k^2)$.

Optimize the model by maximizing the log likelihood function $\log \ p(x \ | \ \pi, \mu, \sigma^2)$ over N pixels through EM algorithm. 

<span class="red">Plot the learning curve for log likelihood of GMM.</span> <span class="blue">(Please terminate EM algorithm when the number of iterations arrives at 100.)</span>

#### 2.3

Repeat steps 1 and 2 for $K = 2, 3, 7$ and $20$. Please show the resulting images of $K$-means model and GMM, respectively. 

Below are some examples.

<center>
    <img src = "./image/figure2.png" width = 50%>
</center>

#### 2.4

Make some discussion about what is crucial factor to affect the output image between $K$-means and Gaussian mixture model (GMM), and explain the reason.

#### 2.5

The input image shown below comes from the licence-free dataset for personal and commercial use. 

Image from: https://pickupimage.com/free-photos/Cat-in-the-forest/2333003

<center>
    <img src = "./image/figure3.png" width = 30%>
</center>